In [1]:
# Data: http://saludata.saludcapital.gov.co/osb/datos_abiertos_osb/enf-transmisibles/OSB_EnfTransm-Uso-UCIS-COVID.csv

In [2]:
import pandas as pd

In [3]:
ucis = pd.read_csv( './data/uci_bogota_raw.csv', encoding = 'ISO-8859-1', sep = ';' )

In [4]:
ucis.shape

(73, 4)

In [5]:
ucis[ 'Fecha' ] = pd.to_datetime( ucis[ 'Fecha' ], format = '%d/%m/%y' )

In [6]:
ucis[ '% Ocupación' ] = ucis[ 'Camas UCI ocupadas Covid-19' ] / ucis[ 'Camas UCI Disponibles COVID 19' ]

In [7]:
ucis.dtypes

Fecha                             datetime64[ns]
Camas UCI ocupadas Covid-19                int64
Camas UCI Disponibles COVID 19             int64
Ocupación UCI COVID 19                    object
% Ocupación                              float64
dtype: object

In [12]:
ucis.tail(20)

,Fecha,Camas UCI ocupadas Covid-19,Camas UCI Disponibles COVID 19,Ocupación UCI COVID 19,% Ocupación
53,2020-05-31,290,670,"43,30%",0.432836
54,2020-06-01,295,670,"44,00%",0.440299
55,2020-06-02,312,670,"46,60%",0.465672
56,2020-06-03,316,670,"47,20%",0.471642
57,2020-06-04,304,674,"45,10%",0.451039
58,2020-06-05,321,712,"45,10%",0.450843
59,2020-06-06,307,716,"42,90%",0.428771
60,2020-06-07,320,716,"44,70%",0.446927
61,2020-06-09,338,722,"46,80%",0.468144
62,2020-06-10,347,716,"48,50%",0.484637


In [10]:
ucis.to_csv( './data/uci_bogota.csv', index = False )